In [1]:
import requests
from bs4 import BeautifulSoup
import html2text
import pandas as pd
import time

In [2]:
def get_url(url,query={}):
    response = requests.get(url, params=query)
    if response.ok :
        #print('OK')
        return BeautifulSoup(response.text, 'html.parser')
    else: print("Failed", response.status_code)

In [18]:
url = 'https://www.indeed.com/jobs'
soup = get_url(url,query)
print(soup.prettify()[:20])

<html>
 <head>
  <ti


In [19]:
print(soup.prettify())

<html>
 <head>
  <title>
   hCaptcha solve page
  </title>
  <script async="" defer="" src="https://www.hcaptcha.com/1/api.js">
  </script>
 </head>
 <body>
  <form action="/jobs?q=Data+scientist&amp;l=United+States&amp;start=585" method="POST">
   <div class="h-captcha" data-sitekey="eb27f525-f936-43b4-91e2-95a426d4a8bd">
   </div>
   <br/>
   <input type="submit" value="Submit"/>
  </form>
 </body>
</html>



In [16]:
def get_indeed(query):
    url = 'https://www.indeed.com/jobs'
    soup = get_url(url,query)
    print(soup.prettify()[:20])
    data = {}
    # GET titles and links
    infos = soup.find_all('a', class_="jobtitle turnstileLink")
    data['title'] = [x['title'] for x in infos]
    data['link'] = [url[:-5]+x['href'] for x in infos]
    
    # GET location
    locations = soup.find_all('div',class_="recJobLoc")
    data['location'] = [x['data-rc-loc'] for x in locations]
    
    # GET posted time
    post_date = soup.find_all('span', class_="date date-a11y")
    dates = [x.text for x in post_date]
    
    # GET company name
    companies = soup.find_all('span',class_='company')
    data['companie'] = [x.text.strip() for x in companies]
    
    # GET job description
    des = []
    for link in data['link']:
        soup0 = get_url(link)
        des.append(soup0.find('div',class_="jobsearch-jobDescriptionText").text)
        time.sleep(0.1)
    data['description'] = des
    
    # return pandas data frame
    return pd.DataFrame(data)

In [4]:
query = {'q':"Data scientist",
         'l':"United States",
         'start':0
        }
df = get_indeed(query)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        15 non-null     object
 1   link         15 non-null     object
 2   location     15 non-null     object
 3   companie     15 non-null     object
 4   description  15 non-null     object
dtypes: object(5)
memory usage: 728.0+ bytes


,title,link,location,companie,description
0,Data Entry,https://www.indeed.com/rc/clk?jk=ce835a296b169...,Remote,TransPerfect Translations Inc.,We are looking for Data Entry Clerks to join o...
1,Data Scientist,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,"Houston, TX","TechniPower, Inc.","I have a client located in Houston, TX that ha..."
2,Data Scientist - Entry Level,https://www.indeed.com/rc/clk?jk=3dde013381898...,"Sacramento, CA",Numerdox,As a Data Scientist you will be working on con...
3,Associate Data Scientist,https://www.indeed.com/rc/clk?jk=9ca3de0881047...,"Deerfield, IL",WALGREENS,Job Summary:\n\nApplies data analysis tools an...
4,"Junior Data Scientist, ML R&D",https://www.indeed.com/rc/clk?jk=f520fca7843c3...,Remote,"LockerDome, Inc.","LockerDome is an ad platform with a brain, des..."


In [13]:
%%time
for i in range(135,600,15):
    print(i)
    query = {'q':"Data scientist",
         'l':"United States",
         'start':i
        }
    df1 = get_indeed(query)
    df = pd.concat((df,df1),ignore_index=True)
    time.sleep(0.5)
title = 'Data Scientist Indeed'
df.to_csv(title+'.csv',index=False)

135
150
165
180
195
210
225
240
255
270
285
300
315
330
345
360
375
390
405
420
435
450
465
480
495
510
525
540
555
570
585
Wall time: 23.1 s


In [14]:
df.shape

(135, 5)

In [17]:
query = {'q':"Data scientist",
     'l':"United States",
     'start':i
    }
df1 = get_indeed(query)

<html>
 <head>
  <ti
